In [1]:
import pandas as pd
import json

In [58]:
url = "https://partners.api.skyscanner.net/apiservices/v3/flights/indicative/search"

payload = { "query": {
        "market": "UK",
        "locale": "en-GB",
        "currency": "EUR",
        "queryLegs": [
            {
                "originPlace": { "queryPlace": { "iata": "LIS" } },
                "destinationPlace": { "queryPlace": { "iata": "LTN" } },
                 "fixedDate": {
            "year": 2023,
            "month": 5,
            "day": 17
        }
            }
        ]
    } }
headers = {
    "x-api-key": "prtl6749387986743898559646983194"
}

response = requests.post(url, json=payload, headers=headers)

print(response.json())

{'status': 'RESULT_STATUS_COMPLETE', 'content': {'results': {'quotes': {'{bl}:202305121806*D*LIS*LTN*20230517*ctuk*W9': {'minPrice': {'amount': '27', 'unit': 'PRICE_UNIT_WHOLE', 'updateStatus': 'PRICE_UPDATE_STATUS_UNSPECIFIED'}, 'isDirect': True, 'outboundLeg': {'originPlaceId': '95565055', 'destinationPlaceId': '95565053', 'departureDateTime': {'year': 2023, 'month': 5, 'day': 17, 'hour': 0, 'minute': 0, 'second': 0}, 'quoteCreationTimestamp': '1683914760', 'marketingCarrierId': '-30972'}, 'inboundLeg': {'originPlaceId': '', 'destinationPlaceId': '', 'departureDateTime': {'year': 0, 'month': 0, 'day': 0, 'hour': 0, 'minute': 0, 'second': 0}, 'quoteCreationTimestamp': '0', 'marketingCarrierId': ''}}}, 'carriers': {'-30972': {'name': 'Wizz Air UK', 'imageUrl': 'https://logos.skyscnr.com/images/airlines/W9.png', 'iata': 'W9'}}, 'places': {'27544008': {'entityId': '27544008', 'parentId': '29475375', 'name': 'London', 'type': 'PLACE_TYPE_CITY', 'iata': 'LON', 'coordinates': {'latitude': 5

In [59]:
def flatten_dict(dd, separator='_', prefix=''):
    return {f"{prefix}{separator}{k}" if prefix else k: v
            for kk, vv in dd.items()
            for k, v in (flatten_dict(vv, separator, kk).items() if isinstance(vv, dict) else {kk: vv}.items())}

In [60]:
output= response.json()
results = output['content']['results']
quotes = results['quotes']
quotes_df = pd.json_normalize(quotes)
quotes_df.columns = [col.split(".")[-1] for col in quotes_df.columns]
flattened_results = [flatten_dict(result) for result in quotes.values()]
quote_df = pd.DataFrame.from_records(flattened_results)
quote_df

,minPrice_amount,minPrice_unit,minPrice_updateStatus,isDirect,outboundLeg_originPlaceId,outboundLeg_destinationPlaceId,outboundLeg_departureDateTime_year,outboundLeg_departureDateTime_month,outboundLeg_departureDateTime_day,outboundLeg_departureDateTime_hour,...,inboundLeg_originPlaceId,inboundLeg_destinationPlaceId,inboundLeg_departureDateTime_year,inboundLeg_departureDateTime_month,inboundLeg_departureDateTime_day,inboundLeg_departureDateTime_hour,inboundLeg_departureDateTime_minute,inboundLeg_departureDateTime_second,inboundLeg_quoteCreationTimestamp,inboundLeg_marketingCarrierId
0,27,PRICE_UNIT_WHOLE,PRICE_UPDATE_STATUS_UNSPECIFIED,True,95565055,95565053,2023,5,17,0,...,,,0,0,0,0,0,0,0,


In [61]:
def flight_days(origin,destination,start_date,end_date):
    url = "https://skyscanner65.p.rapidapi.com/api/v1/flights/indicative/search"
    all_quotes = []
    date_range = pd.date_range(start=start_date, end=end_date, freq='D')
    
    for date in date_range: 
        year = date.year
        month = date.month
        day = date.day          
        payload = { "query": {
                "market": "UK",
                "locale": "en-GB",
                "currency": "EUR",
                "queryLegs": [
                    {
                        "originPlace": { "queryPlace": { "iata": origin } }, 
                        "destinationPlace": { "queryPlace": { "iata": destination } },
                         "fixedDate": {
                    "year": year,
                    "month": month,
                    "day": day
                }
                    }
                ]
            } }
        headers = {
            "content-type": "application/json",
            "X-RapidAPI-Key": "7e479aa293mshe073d8fe00da2c0p1948f7jsnc8a5c51b7f32",
            "X-RapidAPI-Host": "skyscanner65.p.rapidapi.com"
        }
        
        
     
        output= response.json()
        results = output['content']['results']
        quotes = results['quotes']
        quotes_df = pd.json_normalize(quotes)
        quotes_df.columns = [col.split(".")[-1] for col in quotes_df.columns]
        flattened_results = [flatten_dict(result) for result in quotes.values()]
        quote_df = pd.DataFrame.from_records(flattened_results)
        all_quotes.append(quote_df)

        combined_df = pd.concat(all_quotes)
        

    
    return combined_df 

flight_data = flight_days('LIS', 'LTN', '2023-05-17', '2023-06-22')
flight_data

,minPrice_amount,minPrice_unit,minPrice_updateStatus,isDirect,outboundLeg_originPlaceId,outboundLeg_destinationPlaceId,outboundLeg_departureDateTime_year,outboundLeg_departureDateTime_month,outboundLeg_departureDateTime_day,outboundLeg_departureDateTime_hour,...,inboundLeg_originPlaceId,inboundLeg_destinationPlaceId,inboundLeg_departureDateTime_year,inboundLeg_departureDateTime_month,inboundLeg_departureDateTime_day,inboundLeg_departureDateTime_hour,inboundLeg_departureDateTime_minute,inboundLeg_departureDateTime_second,inboundLeg_quoteCreationTimestamp,inboundLeg_marketingCarrierId
0,27,PRICE_UNIT_WHOLE,PRICE_UPDATE_STATUS_UNSPECIFIED,True,95565055,95565053,2023,5,17,0,...,,,0,0,0,0,0,0,0,
0,27,PRICE_UNIT_WHOLE,PRICE_UPDATE_STATUS_UNSPECIFIED,True,95565055,95565053,2023,5,17,0,...,,,0,0,0,0,0,0,0,
0,27,PRICE_UNIT_WHOLE,PRICE_UPDATE_STATUS_UNSPECIFIED,True,95565055,95565053,2023,5,17,0,...,,,0,0,0,0,0,0,0,
0,27,PRICE_UNIT_WHOLE,PRICE_UPDATE_STATUS_UNSPECIFIED,True,95565055,95565053,2023,5,17,0,...,,,0,0,0,0,0,0,0,
0,27,PRICE_UNIT_WHOLE,PRICE_UPDATE_STATUS_UNSPECIFIED,True,95565055,95565053,2023,5,17,0,...,,,0,0,0,0,0,0,0,
0,27,PRICE_UNIT_WHOLE,PRICE_UPDATE_STATUS_UNSPECIFIED,True,95565055,95565053,2023,5,17,0,...,,,0,0,0,0,0,0,0,
0,27,PRICE_UNIT_WHOLE,PRICE_UPDATE_STATUS_UNSPECIFIED,True,95565055,95565053,2023,5,17,0,...,,,0,0,0,0,0,0,0,
0,27,PRICE_UNIT_WHOLE,PRICE_UPDATE_STATUS_UNSPECIFIED,True,95565055,95565053,2023,5,17,0,...,,,0,0,0,0,0,0,0,
0,27,PRICE_UNIT_WHOLE,PRICE_UPDATE_STATUS_UNSPECIFIED,True,95565055,95565053,2023,5,17,0,...,,,0,0,0,0,0,0,0,
0,27,PRICE_UNIT_WHOLE,PRICE_UPDATE_STATUS_UNSPECIFIED,True,95565055,95565053,2023,5,17,0,...,,,0,0,0,0,0,0,0,
